<div style="margin: 0 auto 10px; height: 70px; border: 2px solid gray; border-radius: 6px;">
  <div style="float: left; margin: 5px 10px 5px 10px; "><img src="img/bfh.jpg" /></div>
  <div style="float: right; margin: 20px 30px 0; font-size: 15pt; font-weight: bold; color: #98b7d2;"><a href="https://moodle.bfh.ch/course/view.php?id=39255" style="color: #98b7d2;">BTE5476 - Project-Oriented Digital Signal Processing </a></div>
</div>
<div style="clear: both; font-size: 30pt; font-weight: bold; color: #64788b; margin-left: 30px;">
    DSP with Python, Numpy, and Matplotlib
</div>

Welcome to BTE5476, Project-Oriented Digital Signal Processing. During the semester we will study the details and the implementation of several signal processing algorithms using cases studies from telecommunications, audio processing, and data science. Examples and exercises will be in the form of Jupyter notebooks in Python and we recommend that you review the relevant documentation if you are not familiar with the tools we will use. In particular:

 * [Jupyter notebooks](https://jupyter.org/) are an extremely popular framework for education and code prototyping; the great advantage in using notebooks is that the code, data, and commentary all "live together". You may also want to quickly review the basic syntax of the [markup language](https://commonmark.org/help/tutorial/) used in Jupyter.
 * the two essential Python libraries for this module are [Numpy](https://numpy.org/) and [Matplotlib](https://matplotlib.org/); we will also rely on [SciPy](https://scipy.org/) and on its [signal processing API](https://docs.scipy.org/doc/scipy/reference/signal.html)

To setup your work enviroment, you can download [Anaconda](https://www.anaconda.com/), a bundled installer that contains all the libraries that we will use. Feel free to roll your own Python and Jupyter environment if you prefer, but remember that in this case you will be responsible for any troubleshooting if the class notebooks don't run (they are tested on JupyterLab 4.0.1 using Python 3.12.3).

# Let's get started

The first thing you will find in all notebooks is the inclusion of all necessary libraries:

In [ ]:
# this is a "magic" command use to make plots appear inside the notebook and below the cell with the plotting code
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
# Ipython is used to play audio files
import IPython
# with this we can read sound files in WAV format
from scipy.io import wavfile

In [ ]:
# this sets the default size for the plots
plt.rcParams["figure.figsize"] = (9,2.5)

## Numpy for signal processing

In our notebooks we will use Numpy array to represent digital signals. Here are a few handy things to keep in mind when working with Numpy

In [ ]:
N = 10

# creating data arrays of length N
x = np.zeros(N)
x = np.ones(N)

# from Python lists and tuples to arrays
x = np.array([1, 2, 4.2])
x = np.array((1, 2, 4.2))

# casting to other formats
x = x.astype(np.int32)

# concatenating data
x = np.r_[ np.zeros(2), np.ones(10), np.array([1.1, 1.2, 1.3]) ]

# quick iterations on arrays. For example, set all negative values to zero
x = np.array( [ 0 if t < 0 else t for t in np.random.randn(100) ] )

## Coding style

It's important to write clean code even for short examples. Try to follow the standard [naming conventions](https://peps.python.org/pep-0008/) and to use [type hinting](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html) as much as possible.

With few exceptions, signal processing functions and class methods should operate on arrays and return arrays.

In [ ]:
# example of a function that perfoms sinusoidal modulation on an input signal:
#  y[n] = x[n] cos(w n)

def modulate(x: np.ndarray, w: float) -> np.ndarray:
    return x * np.cos(w * np.arange(0, len(x)))

## Audio processing

Digital audio signals are great to work with because we can listen to the results of our processing algorithms. We will use audio quite a bit in this class so here are some pointers to get started.

SciPy's [input-output module](https://docs.scipy.org/doc/scipy/reference/io.html) provides us with convenient functions to read and write audio files in the uncompressed [WAV format](https://en.wikipedia.org/wiki/WAV):

 * [`scipy.io.wavfile.read()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html#scipy.io.wavfile.read)  loads a WAV file and returns its sampling rate (useful for playback) and the audio data as a NumPy array. The data samples are usually 16-bit signed integers; for convenience, since we're not usually concerned with memory constraints, we will often convert audio data to floating point values between -1 and 1.
 * [`scipy.io.wavfile.write()`](https://docs.scipy.org/doc/scipy/reference/io.html#module-scipy.io.wavfile) to write audio data to disk in WAV format. If your data is in floating point format, remember to convert it back to 16-bit signed ints.

Finally, the IPython package, written by the same authors of the Jupyter framework, is a Python framework geared at interactive and multimedia-based computing; to play audio files it provides the function `IPython.display.Audio()` which returns an interactive widget.

In [ ]:
Fs, s = wavfile.read('data/groove.wav')
print(f'sampling rate: {Fs} Hz, length: {len(s)/Fs} seconds')

In [ ]:
# plot the data with a time scale in seconds
plt.plot(np.arange(0, len(s)) / Fs, s);

In [ ]:
IPython.display.Audio(s, rate=Fs)

# Discrete-Time Signals

## Infinite-length signals

In digital signal processing we deal with objects called _discrete-time signals_. In their most general form, discrete-time (DT) signals are _infinite_ sequences of real- or complex-valued samples and each sample in the sequence is indexed by an integer-valued variable that represents discrete time; in mathematical notation we denote a generic DT signal like so:

$$
    x[n], \quad n \in \mathbb{Z}.
$$

Infinite-length signals are obviously abstract entities but, if the values in the sequence can be expressed in closed form as a function of the time index $n$, we can implement a "signal generator" using Python classes. For instance, a signal of the form

$$
    x[n] = \cos(\omega_0 n + \theta)
$$

can be implemented like so:

In [ ]:
class RealSinusoid:
    def __init__(self, omega: float, theta: float):
        self.omega = omega
        self.theta = theta
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        return np.cos(n * self.omega + self.theta)

Note that $\omega$ is the frequency of the sinusoid (in radians) and $\theta$ is the inital phase. The above class implements a discrete-time _oscillator_ that we can now instantiate with the desired frequency and initial phase to generate any number of samples; note how numpy allows us to apply the cosine function to an entire array of indexes:

In [ ]:
n = np.arange(-100, 100)  # "time" range

osc = RealSinusoid(np.pi/50, 0)
plt.plot(n, osc.samples(n));

# you can also instantiate an object without assignment for one-time use
plt.plot(n, 0.5 * RealSinusoid(np.pi/30, np.pi/4).samples(n), ':');

As another example, here is the generator for a sawtooh signal with unit amplitude and a period of $N$ samples:

In [ ]:
class Sawtooth:
    def __init__(self, period: int):
        self.N = period
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        return np.mod(n, self.N) / (self.N - 1)

In [ ]:
n = np.arange(-100, 100)
saw = Sawtooth(50)
plt.plot(n, saw.samples(n));

If we want to highlight the discrete-time nature of a signal, we can use "lollipop" plots:

In [ ]:
plt.stem(saw.samples(np.arange(0, 100)));

**Exercise: square wave generator.** Complete the code for the generator below so that it produces a unit-amplitude square wave with period $N$ and pulse width $M$ samples.

In [ ]:
class Square:
    def __init__(self, period: int, pulse: int):
        assert pulse <= period, 'duty cycle cannot be greater than one'
        # your code here
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        # your code here
        pass

In [ ]:
# SOLUTION

class Square:
    def __init__(self, period: int, pulse: int):
        assert pulse <= period, 'duty cycle cannot be greater than one'
        self.N = period
        self.cycle = -np.ones(period)
        self.cycle[:pulse] = 1
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        return self.cycle[np.mod(n, self.N)]

In [ ]:
# test
plt.stem(Square(50, 20).samples(np.arange(0, 110)));

## Random sequences

Random signals are another useful type of infinite-length signals, since they can be used to model _noise_. Here is a generator for white Gaussian noise, i.e., a signal where each sample is a random value drawn from a Gaussian distribution with zero mean and variance $\sigma^2$:

In [ ]:
class WGN:
    def __init__(self, sigma: float):
        self.sigma = sigma
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        return np.random.randn(len(n)) * self.sigma

In [ ]:
noise = WGN(1)
plt.plot(noise.samples(np.arange(0, 1000)));

## Finite-length signals

Signal generators are useful but of course in practice we can only process a finite amount of data. A finite-length signal is equivalent to a one-dimensional vector and therefore it is ideally represented by a 1D Numpy array.

We have already implicitly used data arrays before when we created the data for the plots using signal generators but the more interesting situation is when the data that comes from the real world via an Analog-to-Digital (AD) converters. A typical example is audio files, as we have seen in the introduction.

### Energy and power

We will often talk about a signal's energy; for finite-length signals this corresponds to the sum of the square magnitude of the samples
$$
    E_x = \sum_{n=0}^{N-1} |x[n]|^2.
$$

The energy normalized by the signal's length provides the _power_ of the signal, that is, the average energy per sample
$$
    P_x = \frac{1}{N}\sum_{n=0}^{N-1} |x[n]|^2.
$$

**Exercise: energy of waveforms** Compute the energy of $N=100$ samples of the following signals:
 1. a sinusoid of frequency $\omega = 2\pi/N$
 2. a sawtooth of period $N$
 3. a square wave of period $N$ and pulse width $M=N/2$

In [ ]:
# SOLUTION

N = 100
for gen in [RealSinusoid(2 * np.pi/N, 0), Sawtooth(N), Square(N, N//2)]:
    e = np.sum(gen.samples(np.arange(0, N)) ** 2)
    print(f'energy of {gen.__class__.__name__}: {e}')

### Listening to signals

We can pass the output of our signal generators to IPython's sound engine and listen to the different waveforms. Here are a couple of questions to think about:
 * what happens if you change the playback rate `Fs` ?
 * if you order the waveforms in terms of how loud they sound to your ear, do you get the same order as when you rank them by their energy? Why?

In [ ]:
Fs = 16000     # playback rate for the soundcard (samples per second)
S = 2  # length of audio clips in seconds

# same waveforms as before, with period N
N = 100
for gen in [RealSinusoid(2 * np.pi/N, 0), Sawtooth(N), Square(N, N//2)]:
    x = gen.samples(np.arange(0, S * Fs))  # generate data for S seconds
    print(gen.__class__.__name__)
    display(IPython.display.Audio(x, rate=Fs, normalize=False))

# The frequency domain

Every signal "exists" simultaneously in the time domain and in the frequency domain; we can move back and forth between the two domains using the Fourier transform. 

Let's consider a discrete-time signal of length $N$; the data in the signal can be described in two equivalent ways:
 * as a vector $x[n]$ of $N$ sample values in the time domain
 * as a vector $X[k]$ of $N$ Fourier coefficients in the frequency domain

Let's start with an example to see what this means in practice:

**Exercise: sawtooth from sinusoids.** A discrete-time sawtooth signal $s[n]$ of period $N$ samples can be expressed as a linear combination of $2N$ harmonic sinusoids (that is, $N$ sines and $N$ cosines whose frequencies are all multiples of the same fundamental frequency); the expression is 
$$
    s[n] = \frac{1}{2N} \sum_{k=0}^{N-1} a_k \cos(\omega_k\, n) + b_k \sin(\omega_k\, n)
$$
where
$$
    \omega_k = \frac{2\pi}{N}k, \qquad a_k = \begin{cases} N & k = 0 \\ -1 & k \neq 0 \end{cases} \qquad b_k = \begin{cases} 0 & k = 0 \\ \cot(\omega_N\, k/2) & k \neq 0 \end{cases}
$$

Complete the code below so it generates a sawtooth using the sum of sines and cosines.

In [ ]:
class Sawtooth2:
    def __init__(self, period: int):
        self.N = period
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        # your code here
        pass

In [ ]:
# SOLUTION

class Sawtooth2:
    def __init__(self, period: int):
        self.N = period
        
    def samples(self, n: np.ndarray) -> np.ndarray:
        s = np.ones(len(n)) * N
        for k in np.arange(1, N):
            w = 2 * np.pi / N * k
            s -= np.cos(w * n) + np.sin(w * n) / np.tan(w / 2)
        return s / 2 / N

Now let's compare the two implementations: 

In [ ]:
N = 50
n = np.arange(0, 2 * N)
plt.plot(n, Sawtooth(N).samples(n));
plt.plot(n, Sawtooth2(N).samples(n));

This example is just one particular instance of a remarkable general result: _any_ sequence of $N$ samples can be obtained as a linear combination of $2N$ harmonic sinusoids with fundamental frequency $2\pi/N$. Note that a sinusoid with frequency $2\pi k/N$ completes $k$ full periods over $N$ samples and so all sinusoids are $N$-periodic as well.

## The DFT (and FFT)

Since it's messy to carry around both sines and cosines, we use complex exponentials instead.  (always remember Euler's formula, $e^{j\alpha} = \cos\alpha + j\sin\alpha$). The synthesis formula from frequency to time becomes
$$
    x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k]\, e^{j\frac{2\pi}{N}nk} , \quad n=0, \ldots, N-1
$$
and the analysis formula to compute the Fourier coefficients is 
$$
    X[k] = \sum_{n=0}^{N-1} x[n]\, e^{-j\frac{2\pi}{N}nk}, \quad k=0, \ldots, N-1.
$$
The $N$ coefficients $X[k]$ are known as the DFT (Discrete Fourier Transform) of the finite-length signal $x[n]$; they can be computed via an efficient algorithm known as the FFT.

**Example revisited:** Using complex exponentials, it's not difficult to show that the sawtooth signal of period $N$ can be obtained as
$$
    s[n] = \frac{1}{N} \sum_{k=0}^{N-1} \frac{a_k + jb_k}{2}\, e^{j\frac{2\pi}{N}nk} , \quad n=0, \ldots, N-1
$$
Instead of computing the summation explicitly, we can compute all the $N$ values of $s[n]$ at once and very efficiently using Numpy's FFT library:

In [ ]:
def sawtooth_DFT(N : int) -> np.ndarray:
    # returns the DFT coefficients of a sawtooth of period N
    return (np.r_[ N, -np.ones(N-1) ] + np.r_[ 0, 1j /  np.tan(np.pi / N * np.arange(1,N)) ] ) / 2

In [ ]:
# compute and plot one period of the sawtooth from the DFT coefficients
plt.plot(np.real(np.fft.ifft(sawtooth_DFT(50))));

We explicitly select the real part of the inverse DFT because the finite numerical precision of floating point numbers causes the imaginary part to be just very very small instead of exactly zero: 

In [ ]:
plt.plot(np.imag(np.fft.ifft(sawtooth_DFT(50))));

To compute the DFT coefficients numerically from the time-domain data, we apply the DFT analysis formula. Let's compare the DFT coefficients computed numerically via an FFT with their theoretical values:

In [ ]:
N = 50
S_FFT = np.fft.fft(Sawtooth(N).samples(np.arange(0, N)))
S_thr = sawtooth_DFT(N)
plt.subplot(121)
plt.plot(S_FFT.real);
plt.plot(S_thr.real);
plt.subplot(122)
plt.plot(S_FFT.imag);
plt.plot(S_thr.imag);

## Plotting the Spectrum

The DFT _exactly_ decomposes a signal into harmonic oscillations, and no information is lost. The $k$-th DFT coefficient $X[k]$ shows "how much" of the input is provided by the oscillation of frequency

$$
    \omega_k = \frac{2\pi}{N}k.
$$

The magnitude of the coefficient determines the amplitued of the oscillation, whereas its phase determines how the oscillation aligns with the others. Since the energy of a every complex exponential in the DFT is the same, the square mangnitude of the DFT coefficients shows the distribution of the signal's energy in the frequency domain: this is called the **spectrum** of a signal. Often, instead of the square magnitude we will just plot the magnitude of the DFT; this is equivalent to considering the RMS value of a sinusoid.

Let's plot the magnitude spectrum of the sawtooth signal:

In [ ]:
plt.stem(np.abs(sawtooth_DFT(50)));

**Exercise: energy of periodic complex exponentials.** Show numerically that the energy (that is, the sum of the square magnitudes) of $N$ samples of a complex exponential with frequency $2\pi k/N$ is the same for any integer value of $k$ between $0$ and $N-1$

In [ ]:
# SOLUTION

N = 10
print([ np.sum(np.abs(np.exp(2j * np.pi / N * k * np.arange(0, N))) ** 2) for k in np.arange(0, N) ])

### Positive and negative frequencies

Because of the $2\pi$-periodicity of complex exponentials, a frequency $\omega$ between $\pi$ and $2\pi$ is equivalent to a _negative_ frequency of $\omega - 2\pi$; if we imagine the oscillation as the position of a point rotating in a circular pattern, positive and negative frequencies correspond to counterclockwise and clockwise rotations, respectively. Two frequencies with equal magnitude but opposite sign indicate two rotations with the same angular velocity but with opposite "spin". 

When we consider the full set of $N$ DFT coefficients, the second half of the coefficients can thus be associated to negative frequencies of the form

$$
    \omega_k = -\frac{2\pi}{N}(N - k) \quad \text{for } k \ge L = \lceil N / 2 \rceil.
$$

Because of this, it often makes more sense to rearrange the DFT so that the second half of the coefficients comes before the first and to plot the coefficients centered around zero with positive frequencies on the right and negative frequencies on the left. This is also consistent with the way other types of spectra are usually plotted, as in the case of the frequency response of filters.

Finally, remember that if the original signal is real-valued then the magnitude of the DFT is symmetric around its midpoint, so that in practice we  need to plot only the first half of the DFT coefficients. The same holds if we are interested in the phase of the DFT which, for real-valued inputs, is antisymmetric around its midpoint.

### Centering the DFT around zero

There is an important detail that we must take into account when rearranging the DFT coefficients, namely, *we need to differentiate between odd- and even-length signals*. Indeed, assume the signal (and the DFT) are of length $N$; if we want to rearrange the DFT coefficients so that $X[0]$ becomes the center point

 * For *odd-length* signals, we can write $N = 1 + 2L$ with $L = (N-1)/2 \in \mathbb{N}$, and so the maximum positive frequency corresponds to the index $k=L$ while the minimum negative frequency corresponds to index $k=L+1$; with this:
   * the $L$ coefficients $X[1]$ to $X[L]$ correspond to the positive frequencies ranging from $\omega_1 = 2\pi/N$ to $\omega_L = \pi - \pi/N$
   * the $L$ coefficients $X[L+1]$ to $X[N-1]$ correspond to the negative frequencies ranging from $\omega_{-L} = -\pi + \pi/N$ to $\omega_{-1} = -2\pi/N$ 
 
 * For *even-length* signals, $N = 2L$, with $L = N/2 \in \mathbb{N}$ and so the maximum positive frequency _and_ the minumum negative frequency are _both_ associated to index $k=L$; indeed, $\omega_{L} = \frac{2\pi}{N}\frac{N}{2} = \pi$ and therefore $|\omega_L| = |\omega_L - 2\pi| = \pi$. We therefore need to reuse coefficient number $L$ as the value associated to the minimum negative frequency:
   * the $L$ coefficients $X[1]$ to $X[L]$ correspond to the positive frequencies ranging from $\omega_1 = 2\pi/N$ to $\omega_L = \pi$
   * the $L$ coefficients $X[L]$ to $X[N-1]$ correspond to the negative frequencies ranging from $\omega_{-L} = -\pi$ to $\omega_{-1} = -2\pi/N$ 

With this strategy, we obtain a symmetric DFT magnitude data set in both cases:
 * if $N$ is *odd*, the re-arranged DFT data will be of length $N$ 
 * if $N$ is *even*, the re-arranged DFT data will be of length $N+1$  

**Exercise: rearranging the DFT data.** Complete the code below to implement the centering strategy described above.

In [ ]:
def dft_center(X: np.ndarray) -> np.ndarray:
    pass

In [ ]:
# SOLUTION

def dft_center(X: np.ndarray) -> np.ndarray:
    N, L = len(X), len(X) // 2
    if (N % 2 == 0):
        return np.r_[ X[L:], X[:L+1] ]
    else:
        return np.r_[ X[L+1:], X[:L+1] ]

Let's test your solution:

In [ ]:
for N in [50, 51]:
    X = dft_center(sawtooth_DFT(N))
    assert np.allclose(np.sum(X[:len(X)//2] - np.conj(X[:len(X)//2:-1])), 0), 'sorry, try again'
    plt.figure();
    plt.stem(np.abs(X));

**Exercise: signed indexes.** In order to _plot_ the centered DFT coefficients symmetrically around zero, we need to provide the plot function with an array of signed index between $-L$ and $L$, with $L=\lceil N / 2 \rceil$. Modify the previous function so that it returns both the signed indexes and the rearranged data.

In [ ]:
def dft_center(X: np.ndarray) -> (np.ndarray, np.ndarray):
    pass

In [ ]:
# SOLUTION 

def dft_center(X: np.ndarray) -> (np.ndarray, np.ndarray):
    L = len(X) // 2
    # using more compact code to rearrange the DFT data:
    return np.arange(-L, L + 1), np.r_[ X[L+(len(X) % 2):], X[:L+1] ]

Let's test:

In [ ]:
for N in [50, 51]:
    ix, X = dft_center(sawtooth_DFT(N))
    assert np.allclose(np.sum(X[:len(X)//2] - np.conj(X[:len(X)//2:-1])), 0), 'sorry, try again'
    plt.figure();
    plt.stem(ix, np.abs(X));

**Exercise: signals in the frequency domain.** Using the generators developed so far, plot the zero-centered magnitude spectra of the following signals:
 * 100 samples of a sinusoid with frequency $\omega = \pi/10$
 * 100 samples of a sinusoid with frequency $\omega = \pi/11$
 * 100 samples of a square wave with period $N=50$ and pulse width $M=25$

In [ ]:
# SOLUTION 

for gen in [ RealSinusoid(np.pi/10, 0), RealSinusoid(np.pi/11, 0), Square(50, 25) ]:
    plt.figure();
    plt.stem(*dft_center(np.abs(np.fft.fft(gen.samples(np.arange(0, 100))))));

## Spectral analysis of real-world data

When dealing with analog physical systems, time is measured in seconds (s) and frequencies are measures in hertz (Hz) a unit whose dimensionality is $[s^{-1}]$. In digital signal processing we have access to sampled versions of the analog signal, where "time" is an integer index without physical dimensions; similarly, when we compute the DFT, the frequency index $k$ also is dimensionless. 

If we know the sampling rate of the signal, however, we can map the DFT index to the underlying physical frequencies in Hz: 
 * if the sampling rate is $F_s$ samples per second, then the time interval between discrete-time signal samples is $T_s = 1/F_s$ seconds
 * the $k$-th DFT coefficient $X[k]$ corresponds to a digital frequency $\omega_k = 2\pi k /N$, that is, to a discrete-time oscillation that completes $k$ full cycles over $N$ samples
 * if the time between successive samples is $T_s$, then the $k$-th oscillation completes $k$ cycles over $NT_s$ seconds, that is, its period is $NT_s/k$ seconds
 * in conclusion, the real-world frequency associated to $X[k]$ is $f_k = (k/N)F_s$ Hz.
 
The mapping between DFT indexes and real-world frequenc is thus linear; its factor $\Delta_f = F_s/N$ represents the maximum frequency resolution provided by a DFT of length $N$ for a signal at sampling frequency $F_s$.

### Mapping frequencies

Here is a function that returns a zero-centered DFT together with the frequency values in Hz for each coefficient; if `positive_only` is `True` then only the positive half of the spectrum is returned, which is convenient for real-valued signals whose magnitude spectrum is symmetric.

In [ ]:
def dft_map(X: np.ndarray, Fs: float, positive_only = False) -> (np.ndarray, np.ndarray):
    L = len(X) // 2
    ix, S = np.linspace(-Fs/2, Fs/2, 2*L+1), np.r_[ X[L+(len(X) % 2):], X[:L+1] ]
    if positive_only:
        return ix[-L-1:], S[-L-1:]
    else:
        return ix, S

In [ ]:
plt.stem(*dft_map(np.abs(np.fft.fft(Square(10, 5).samples(n))), 1000, False));

### The pitch of a musical note

Sounds that contain a periodically repeating pattern evoke in a listener the sensation of musical _pitch_; waveforms with a pattern that repeats quickly are associated to notes in the so-called high register, whereas slowly repeating patterns produce notes in the low register. Here is a note played by a piano; if we zoom in we can see the periodicity of the waveform

In [ ]:
Fs, x = wavfile.read("data/piano.wav")
print(f'the audio sampling rate is {Fs} Hz')
IPython.display.Audio(x, rate=Fs)

In [ ]:
plt.plot(x[10000:10500]);

A signal in which a pattern repeats $f_0$ times per second has a spectrum characterized by clear peaks at integer multiples of $f_0$; we call this a _harmonic spectrum_ and $f_0$ is called the _fundamental frequency_ of the musical note. 

To determine $f_0$ we need to locate the first spectral peak with frequency greater than zero; let's begin by looking at the full harmonic structure (we just need the positive frequencies):

In [ ]:
# we don't need to compute the DFT of the whole signal, given its size
ix, X = dft_map(np.abs(np.fft.fft(x[:32768])), Fs, True)
plt.plot(ix, X);

To determine the pitch, we can zoom in and see that the the first peak is in the vicinity of 200Hz; we can zoom in further and find the fundamental frequency which turns out to be about 220 Hz (corresponding to an A played in the 3rd octave)

In [ ]:
plt.plot(ix[10:1000], X[10:1000]);

# Digital filters

For our purposes, a filter is an _algorithm_ that produces one output sample $y[n]$ for every new input sample $x[n]$; in the most general case, therefore, a filter is defined by the sequence of operations used to produce the output. A filter usually contains memory buffers to store and access previous input and output values.

## LTI filters

If the filter's algorithm is both linear and time-invariant (LTI), then we can characterize the filter in several equivalent ways:
 1. by describing its algorithm which, for LTI filters, is always of the form $$
    y[n] = \sum_{k=0}^{N} b_k x[n-k] - \sum_{k=1}^{M} a_k y[n-k];
$$ This is called a Constant-Coefficient Difference equation (CCDE).
 1. by describing its impulse response $h[n]$, that is, the output of the filter when the input is the sequence $$
        \delta[n] = \begin{cases} 1 & n=1 \\ 0 & n \neq 0 \end{cases}.
    $$ Once we have the impulse response, we can compute the filter's output for any input signal $x[n]$ via a convolution: $$
      y[n] = (x \ast h)[n] = \sum_{k=-\infty}^{\infty}h[k]x[n-k]
    $$
 1. by means of the filter's transfer function, which is the $z$-transform of its impulse response and which, for LTI systems, can be obtained directly from the coefficients in the CCDE: $$
        H(z) = \frac{b_0 + b_1 z^{-1} + \ldots + b_{N}z^{-N}}{1 + a_1 z^{-1} + \ldots + a_{M}z^{-M}};
$$
 1. by means of its frequency response, which is the Fourier transform of the impulse response. The frequency response tells us which frequencies will be "killed" by the filter and which frequencies will "survive" and allows us to classify a filter as lowpass, highpass etc.

### Two simple but useful filters

Two simple filters that you should be familiar with are the following:

 1. The **Moving Average** of length $M$ is a lowpass filter described by the CCDE $$
    y[n] = \frac{1}{M}\sum_{k=0}^{M-1}x[n-k] 
    $$ or by the transfer function $$
    H(z) = \sum_{k=0}^{M-1} \frac{1}{M}z^{-k}
    $$ The Moving Average is an FIR filter, that is, each output sample is computed using only past _input_ samples. Its impulse response is $$
    h[n] = \begin{cases} 1/M & 0 \le n < M \\ 0 & \mathrm{otherwise} \end{cases}. $$
 1. The **Leaky Integrator** is a lowpass filter described by the CCDE $$
    y[n] = \lambda y[n] + (1-\lambda)x[n], \qquad 0 < \lambda < 1
    $$ or by the transfer function $$
    H(z) = \frac{1-\lambda}{1-\lambda z^{-1}}
    $$ It is a recursive (IIR) filter and its output at time $n$ depends on the output at time $n-1$; assuming zero initial conditions, that is, everything is zero for $n<0$, we can compute its impulse response $$
    h[n] = \begin{cases} (1-\lambda)\lambda^n & n \ge 0 \\ 0 & n < 0 \end{cases}. $$


### Frequency response

If you know the transfer function of a filter, you can compute its frequency response by taking the ratio of the DFTs of the $b_k$ and the $a_k$ coefficients. Both DFTs should have the same lenght and, to have a smooth plot, usually we zero-pad them to a large number. Numpy's `fft` function can do this for us if we pass the desired length as an optional argument:

In [ ]:
def plot_mag_resp(b: np.ndarray, a: np.ndarray, N: int = 1024) -> None:
    _, H = dft_center(np.abs(np.fft.fft(b, N) / np.fft.fft(a, N)))
    # we rescale the frequency axis to the [-pi, pi] interval
    plt.plot(np.linspace(-np.pi, np.pi, len(H)), H)

Here is the magnitude response of a Moving Average:

In [ ]:
M = 20
plot_mag_resp(np.ones(M)/M, [1])

and here is the magnitude response of a Leaky Integrator:

In [ ]:
lam = 0.9
plot_mag_resp([1 - lam], [1, -lam])

When we know the sampling rate, we may want to plot the frequency response over the actual frequency range of the signal:

In [ ]:
def plot_mag_resp_hz(b: np.ndarray, a: np.ndarray, Fs: float, N: int = 1024) -> None:
    plt.plot(*dft_map(np.abs(np.fft.fft(b, N) / np.fft.fft(a, N)), Fs, True))

## Implementing filters in Python

Although ready-made algorithms are available in SciPy, it's instructive to try and implement LTI filters from scratch to understand the underlying computational requirements. 

### Filters as functions

When filters are implemented as standalone pure functions, the entire input signal is passed as an argument to the function, and the entire output signal is returned. 

As a first example, consider this implementation of the Leaky Integrator:

In [ ]:
def leaky(x: np.ndarray, lam: float) -> np.ndarray:
    y = np.zeros(len(x))
    for n in range(0, len(x)):
        y[n] = lam * y[n-1] + (1 - lam) * x[n]        
    return y

We can verify that the code works by computing the response to an impulse and comparing the output to the theoretical values:

In [ ]:
N = 100
lam = 0.95
plt.stem(leaky(np.r_[1, np.zeros(N-1)], lam));
plt.plot((1 - lam) * lam ** np.arange(0, N), 'C2');

When we call the function we assume zero initial conditions, that is, we assume that $y[n]=0$ for $n<0$. In practice, we need only the value $y[-1]$ in the first iteration and in the implementation we exploit Python's negative indexing feature:
 * we pre-allocate an output array `y` with the same length as the input signal, and we initialize it with zeros
 * in the first iteration of the filtering loop, when `n` is zero, the expression `y[n-1]` becomes `y[-1]` which is the last element of the zero-filled array `y`; therefore `y[n-1]` is indeed equal to zero for `n=0`. 

**Exercise: implement the Moving Average** Implement a MA filter of length $M$ as a function; it should return the output values for $n = 0, 1, \ldots, N-1$, where $N$ is the length of the input sequence, and assume zero initial conditions for the filter (which, in this case, corresponds to assuming $x[n] = 0$ for $n < 0$).

In [ ]:
def ma(x: np.ndarray, M: int) -> np.ndarray:
    ...        
    return y

In [ ]:
# SOLUTION

def ma(x: np.ndarray, M: int) -> np.ndarray:
    N = len(x)
    y = np.zeros(N)
    # prepend M-1 zeros to x so that we can always "go back" M samples to compute the average
    x = np.r_[np.zeros(M-1), x]
    for n in range(0, N):
        # we can use the built-in averaging function in numpy over the previous M samples
        y[n] = np.mean(x[n:n+M])        
    return y

Let's test your implementation:

In [ ]:
y = ma((-1) ** np.arange(0, 40), 20) # test signal, filtered
print('good job!' if np.sum(y[1::2]) == 0 and np.sum(y[:20:2]) == 0.5 else 'Sorry, try again!')

### Filters as classes

Digital filters compute each output sample as a linear combination of past input and output samples. In function-based implementations the _entire_ input is passed as an argument and therefore the "past" is accessible via simple array indexing.

In many signal processing applications, however, a filter needs be used iteratively on the input one or a few samples at a time and, in this case, a function-based implementation is not going to work because variables in [pure functions](https://en.wikipedia.org/wiki/Pure_function) are not persistent across calls.

Remember that the $z^{-1}$ delay elements in a filter's transfer function represent the memory cells that a filter needs to "remember" past input and output samples; at any given point in time, the content of these delay elements contain the _internal state_ of the filter, which must be preserved across successive calls. 

A code object that preserves its internal state across calls is known as _stateful_ and the natural way to implement stateful objects in Python is via [classes](https://www.learnpython.org/en/Classes_and_Objects). 

Here is a stateful Leaky Integrator:

In [ ]:
class LI:
    def __init__(self, lam: float):
        self.y_prev = 0
        self.lam = lam
        
    def reset(self):
        # leaky integrator needs just one memory cell
        self.y_prev = 0
        
    def filt(self, x: np.ndarray) -> np.ndarray:
        y = np.r_[np.zeros(len(x) - 1), self.y_prev]
        for n in range(0, len(x)):
            y[n] = self.lam * y[n-1] + (1 - self.lam) * x[n]
        self.y_prev = y[-1]
        return y

Here too we exploit Python's negative indexing feature; the line 

`y = np.r_[np.zeros(len(x) - 1), self.y_prev]` 

ensures that the element `y[-1]` contains the last output value computed in the previous call to `filt` (or zero if it's the first time that `filt` is called). We also added a "reset" method to clear the internal memory.

In [ ]:
x = np.ones(50)
# create a leaky integrator filter 
li = LI(0.95)

# filtering the whole input
y1 = li.filt(x)
# filtering the input one sample at a time
li.reset()
y2 = [li.filt([x_n]) for x_n in x]

plt.stem(y1);
plt.stem(y2);

**Exercise: a stateful Moving Average**  Complete the `filt` method so that the following class implements a stateful Moving Average filter.

In [ ]:
class MA:
    def __init__(self, M: int):
        # moving average needs to keep track of previous M-1 input samples
        self.buf = np.zeros(M-1)
        self.M = M
        
    def reset(self):
        self.buf = self.buf * 0
        
    def filt(self, x: np.ndarray) -> np.ndarray:
        x = np.array([x]) if np.isscalar(x) else x
        
        ... # your code here
        
        return y

In [ ]:
# SOLUTION

class MA:
    def __init__(self, M: int):
        self.buf = np.zeros(M-1)
        self.M = M
        
    def reset(self):
        self.buf = self.buf * 0
        
    def filt(self, x: np.ndarray) -> np.ndarray:
        N = len(x)
        y = np.zeros(N)
        x = np.r_[self.buf, x]
        for n in range(0, N):
            y[n] = np.mean(x[n:n+self.M])        
        self.buf = x[-self.M+1:]
        return y

Let's test your implementation:

In [ ]:
# create the moving average filter 
ma = MA(11)

# filtering the whole input
y1 = ma.filt(x)
# filtering the input one sample at a time
ma.reset()
y2 = [ma.filt([x_n]) for x_n in x]

plt.stem(y1);
plt.stem(y2);

**Example: stateful implementation of a generic transfer function**
Study the following stateful implementaton of a generic IIR filter with difference equation
$$
    y[n] = \sum_{k=0}^{N-1} b_k x[n-k] - \sum_{k=1}^{M-1} a_k y[n-k].
$$ 

In [ ]:
class DTFilter:
    def __init__(self, B: np.ndarray, A: np.ndarray):
        # normalize (if a_0 != 1) and store coefficients in reversed order
        self.b = np.array(B)[::-1] / A[0]
        self.a = np.array(A)[::-1] / A[0] 
        # allocate buffer for both input and output past values
        self.buf_len = max(len(self.a), len(self.b))
        self.buf = np.zeros((2, self.buf_len))

    def reset(self):
        self.buf = self.buf * 0.0

    def filt(self, data: np.ndarray) -> np.ndarray:
        data_len = len(data)
        self.buf = np.concatenate((self.buf, [data, np.zeros(data_len)]), axis=1)
        # fill the second row: output values
        for n in range(self.buf_len + 1, self.buf_len + data_len + 1):
            self.buf[1][n-1] = self.b @ self.buf[0][n-len(self.b):n] - self.a @ self.buf[1][n-len(self.a):n]
        # extract the last output values to return, the length of which is the length of the input
        y = self.buf[1][-1] if data_len == 1 else self.buf[1][-data_len:]
        # update the buffer
        self.buf = self.buf[:,-self.buf_len:]
        return y

We can check the generic implementation against our previous implementations of the Leaky Integrator and the Moving Average:

In [ ]:
x = np.random.rand(100) - 0.5

lam = 0.9
assert np.allclose(LI(lam).filt(x), DTFilter([1-lam], [1, -lam]).filt(x))

M = 11
assert np.allclose(MA(M).filt(x), DTFilter(np.ones(M) / M, [1]).filt(x))

## Using SciPy

The [SciPy library](https://scipy.org/) is a Python library implementing _"fundamental algorithms for scientific computing"_; among the various subpackages included in SciPy, the [signal processing module](https://docs.scipy.org/doc/scipy/reference/signal.html#module-scipy.signal) provides us with a rich set of ready-made routines. Here are some useful pointer for later

### Filter design

Scipy provides all the fundamental filter design routines; the most useful ones are:

 * [`scipy.signal.butter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html#scipy.signal.butter) to design Butterworth filters (monotonic frequency response)
 * [`scipy.signal.ellip()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ellip.html#scipy.signal.ellip) for elliptical filters (sharp transition bands)
 * [`scipy.signal.remez()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.remez.html#scipy.signal.remez) to design linear-phase optimal FIR filters using the Parks-McClellan algorithm

### Filter analysis and implementation

You may want to familiarize yourself with the following functions:

 * `scipy.signal.freqz()`  to plot the frequency response of a filter [(documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html#scipy.signal.lfilter)
 * `scipy.signal.lfilter()` to filter a signal [(documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html#scipy.signal.lfilter)

## Audio processing examples

We can use digital filters to extract different frequency regions of an audio file. Let's load one of the samples available with this notebook (but you can of course use your own music!)

In [ ]:
Fs, s = wavfile.read('data/sm.wav')
print(f'sampling rate: {Fs} Hz, length: {len(s)/Fs} seconds')
IPython.display.Audio(s, rate=Fs)

### Extracting the low frequencies

Suppose we are interested in hearing more clearly the bass part; since a bass guitar plays notes with pitch from about 40 to 400 Hz, we could try to filter the audio with a lowpass with cutoff 200Hz to focus on the range of the instrument.

Let's start with an elliptic filter for which we can use the `sp.ellip` design routine. Let's first convert the desired cutoff frequency in Hertz to a normalized value that the function understands; according to [the specs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ellip.html#scipy.signal.ellip), the cutoff frequency should be normalized so that a value of 1 corresponds to half the sampling frequency of the signal. 

We are using a 8-th order filter with 60dB attenuation in stopband and 10% max ripple. Feel free to play with these design parameters and see the effects that they have on the output.

In [ ]:
fc = 200  # cutoff frequency in Hz
wc = fc / (Fs / 2)
b, a = sp.ellip(8, .1, 60, wc)

In [ ]:
# let's have a look at the frequency response
plot_mag_resp_hz(b, a, Fs)

In [ ]:
# let's filter the audio and play it
s_filt = sp.lfilter(b, a, s)
IPython.display.Audio(s_filt, rate=Fs)

### Extracting the treble

Just as we tried to extract the bass, we can try to extract parts of the drum pattern. Usually, we get a good feel for the hi-hat and cymbals by keeping frequencies above 4KHz so we will need a highpass filter.

Let's try with an FIR; note that, to design a highpass, we must choose an odd number of taps in order to obtain a type I filter (symmetric with integer delay):

In [ ]:
# cutoff frequency
fc = 4000
# transition band:
tb = 100
# length of the filter 
M = 1001

h = sp.remez(M, [0, fc - tb, fc, Fs/2], [0, 1], weight=[1, 1], Hz=Fs)
plot_mag_resp_hz(h, [1], Fs)

In [ ]:
s_filt = sp.lfilter(h, [1], s)
IPython.display.Audio(s_filt, rate=Fs)